In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU

import keras_tuner
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping, Callback

!pip install shap
import shap
import pickle
import copy

import seaborn as sns; sns.set_theme()
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib as mlp

import lime
import lime.lime_tabular

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [2]:
inp = pd.read_csv("/kaggle/input/ccs-position-data/CCS.csv", header = 0, low_memory=False, sep=",")
inp.index = range(len(inp))
inp.head()

inp = inp.groupby('Sequence', as_index=False).mean()
inp

,Sequence,CCS
0,AAAALEENER,334.0
1,AAAANLNYIR,337.0
2,AAADEWDER,334.0
3,AAADLDVR,296.0
4,AAADLISR,303.0
...,...,...
45985,YYVNSLQHR,375.0
45986,YYYDGDMICK,400.0
45987,YYYFSPLYR,372.0
45988,YYYIPQYK,373.0


In [9]:
#Tạo list các axit amin để encoding
peptides = inp['Sequence'].tolist()
peptides.sort()
vocab = set(''.join([str(i) for i in peptides]))
vocab.add('END')
vocab_list = list(vocab)
vocab_list.sort()
#vocab_list

#Thêm END vào các peptides dài 8-9 có độ dài 10 
char_index = dict((vocab_list[i], i) for i in range(len(vocab_list)))
X = []
x_name = [str(i)[0:10] for i in peptides]
for i in x_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,10 - len(str(i))):
        tmp.append(char_index["END"])
    X.append(tmp)
#X

#Tạo list chứa toàn bộ giá trị CCS
Y = []
for index, row in inp.iterrows():
    my_list = [row.CCS]
    Y.append(my_list)
#all_int_list[0]

X = np.asarray(X)
Y = np.asarray(Y)

In [ ]:
#Chia thành tập train, test, val
x_train_all, x_test, y_train_all, y_test = train_test_split(X,Y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=42)

In [ ]:
np.savetxt('CCS_x_test.txt', x_test)
np.savetxt('CCS_y_test.txt', y_test)
np.savetxt('CCS_x_train.txt', x_train)
np.savetxt('CCS_y_train.txt', y_train)
np.savetxt('CCS_x_val.txt', x_val)
np.savetxt('CCS_y_val.txt', y_val)

In [4]:
x_train = np.loadtxt('/kaggle/input/ccs-position-data/CCS_x_train.txt')

x_test = np.loadtxt('/kaggle/input/ccs-position-data/CCS_x_test.txt')

y_train = np.loadtxt('/kaggle/input/ccs-position-data/CCS_y_train.txt')

y_test = np.loadtxt('/kaggle/input/ccs-position-data/CCS_y_test.txt')

x_val = np.loadtxt('/kaggle/input/ccs-position-data/CCS_x_val.txt')

y_val = np.loadtxt('/kaggle/input/ccs-position-data/CCS_y_val.txt')

In [ ]:
%%time

#Model LSTM 
model = Sequential()
model.add(Embedding(output_dim = 50, input_dim = 21, input_length = 10))

model.add(LSTM(128,return_sequences=False, input_shape=(10,21)))
model.add(Dropout(0.5324275624952207))

model.add(Dense(64))
model.add(Dropout(0.08657063211846627))

model.add(Dense(1))

optimizermodel = tf.keras.optimizers.Adam(0.001)
optimizermodel.learning_rate.assign(0.005)
model.compile(loss='mse', optimizer = optimizermodel, metrics=['mse'])

hist = model.fit(x_train, y_train,
                batch_size = 128,
                epochs = 200,
                validation_data = (x_val, y_val))



In [ ]:
y_pred = model.predict(x_test)
model_r2 = rs_score(y_test, y_pred)
model_mae = mean_absolute_error(y_test, y_pred)
model_mse = mean_squared_error(y_test, y_pred)
print(model_r2, model_mae, model_mse)

In [44]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(output_dim=50, input_dim=21, input_length=10))

    model.add(GRU(hp.Int('units_1', min_value=32, max_value=128, step=32),
                  return_sequences=True, input_shape=(10, 21)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.8)))

    model.add(GRU(hp.Int('units_2', min_value=32, max_value=128, step=32)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.8)))

    model.add(Dense(1))

    learning_rate = hp.Choice('learning_rate', values=[0.001, 0.005])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_mse',
    max_trials=10,  
    executions_per_trial=1,  
    directory='tuner_results',
    project_name='gru_model_tuning_9'
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

tuner.search(x_train, y_train,
             epochs=200,
             batch_size=tuner.oracle.hyperparameters.Choice('batch_size', values=[64, 128, 256, 512]),
             validation_data=(x_val, y_val),
             callbacks=[early_stopping],
             verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]
best_run = tuner.oracle.get_best_trials(num_trials=1)[0]

best_model.summary()
print(best_run.hyperparameters.values)


Trial 10 Complete [00h 03m 41s]
val_mse: 171.72860717773438

Best val_mse So Far: 171.72860717773438
Total elapsed time: 00h 32m 19s


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 10, 50)         │         1,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 10, 128)        │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 96)             │        65,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,355 (528.73 KB)

 Trainable params: 135,355 (528.73 KB)

 Non-trainable params: 0 (0.00 B)

144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 180.6588 - mse: 180.6588
Test MSE: 170.21998596191406


In [49]:
%%time

#model GRU
model1 = Sequential()
model1.add(Embedding(output_dim = 50, input_dim = 21, input_length = 10))

model1.add(GRU(128, return_sequences=True))
model1.add(Dropout(0.7629949170837947))

model1.add(GRU(64))
model1.add(Dropout(0.05120694424276691))

model1.add(Dense(1))

optimizer = tf.keras.optimizers.Adam(0.001)

model1.compile(loss='mse', optimizer=optimizer)
model1.fit(x_train, y_train, 
           epochs = , 
           batch_size = 64, 
           validation_data = (x_val, y_val))

Epoch 1/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 107888.0156 - val_loss: 83753.6953
Epoch 2/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 77043.1172 - val_loss: 59614.7305
Epoch 3/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 54512.8906 - val_loss: 41000.5938
Epoch 4/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 37178.1133 - val_loss: 26970.0449
Epoch 5/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 24102.8203 - val_loss: 16760.2734
Epoch 6/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 14781.0166 - val_loss: 9720.0664
Epoch 7/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 8487.6719 - val_loss: 5212.4258
Epoch 8/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 4497.3799 - val_loss: 2611.7898
Epoch 9/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 2272.4597 - val_loss: 1326.4073
Epoch 10/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 1233.8921 - val_loss: 810.4675
Epoch 11/100
518/518 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - lo

In [51]:
y_pred1 = model1.predict(x_test)
model1_r2 = r2_score(y_test, y_pred1)
model1_mae = mean_absolute_error(y_test, y_pred1)
model1_mse = mean_squared_error(y_test, y_pred1)
print(model1_r2, model1_mae, model1_mse)

144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
0.6702093586866547 9.812438503662694 200.8540608978574


In [ ]:
#Chọn model
model = model1

**SHAP**

In [ ]:
%%time

#Tạo 100 cụm dữ liệu để tính toán SHAP
x_trainmeans = shap.kmeans(np.asarray(x_train), 100)
explainer = shap.KernelExplainer(model.predict, x_trainmeans)
shap_values = explainer.shap_values(np.asarray(x_test))

#lưu giá trị SHAP
with open("CCS_shapvalues_GRU.pkl","wb") as f:
    pickle.dump(shap_values, f)

In [ ]:
with open("/kaggle/input/ccs-position-data/shapvaluesCCS (1).pkl","rb") as f:
    shap_values = pickle.load(f)

**LIME**

In [ ]:
%%time

#Định nghĩa hàm giải thích LIME
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data = x_train,
    mode = 'regression',
    feature_names = [f'Pos{i+1}' for i in range(10)],
)
positions = [f'Pos{i+1}' for i in range(10)]

lime_values = []
for i in range(len(x_test)):
    x_instance = x_test[i]
    ordered_contributions = {pos: 0.0 for pos in positions}

    #Tính LIME cho từng instance
    exp = explainer.explain_instance(
        data_row = x_instance,
        predict_fn = model.predict
    )

    for feature_value, contribution in exp.as_list():
        pos = next((word for word in feature_value.split() if word.startswith('Pos')), None)
        if pos in ordered_contributions:
            ordered_contributions[pos] += contribution

    contributions_sorted = [ordered_contributions[pos] for pos in positions]
    lime_values.append(contributions_sorted)

with open('CCS_lime_values_LSTM.pkl', 'wb') as f:
    pickle.dump(lime_values, f)

In [ ]:
with open('/kaggle/input/ccs-position-data/CCS_lime_values_LSTM.pkl', 'rb') as f:
    lime_values = pickle.load(f)

**OSA**

In [ ]:
%%time

#Tính OSA
baseline = 0
original_output = model.predict(x_test).flatten()
osa_values = np.zeros_like(x_test, dtype=np.float32)

for i in range(len(x_test)):
    osa_inputs = x_test.copy()
    osa_inputs[:,i] = baseline
    osa_output = model.predict(osa_inputs).flatten()

    osa_values[:,i] = np.abs(original_output - osa_output)



**HEATMAP**

In [ ]:
#Tạo mange lưu các giá trị SHAP/LIME/OSA của 1 axit amin ở cùng 1 vị trí (pos=[1,10])

#Chọn giá trị
explain_values = shap_values

aa = []
i = 0
while i < 10
    aa.append([[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]])
    i+=1

i=0
while i < len(x_test):
    j = 0
    while j < len(x_test[i]):
        aa[j][int(x_test[i][j])].append(explain_values[0][i][j])
        j+=1
    i+=i

heatmap = []

i = 0
while i < len(aa):
    j = 0
    heatmap.append([])
    while j < len(aa[i]):
        if len(aa[i][j]) > 1:
            aa[i][j] = aa[i][j][1:]
            heatmap[i].append(sum(aa[i][j])/float(len(aa[i][j])))
        else:
            heatmap[i].append(0)
        j+=1
    i+=1
heatmap = np.array(heatmap)

In [ ]:
plt.figure()
x_axis_labels = ['A','C','D','E','End','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'] # labels for x-axis
y_axis_labels = ['1','2','3','4','5','6','7','8','9','10']
sns.set(font_scale = 2)

rcParams['figure.figsize'] = 20,16
ax = sns.heatmap(heatmap,xticklabels=x_axis_labels, yticklabels=y_axis_labels,linewidths=.5,  cmap="viridis")

ax.set(xlabel='Amino Acid', ylabel='Position', title='Mean SHAP Values - CCS')
ax.figure.savefig('CCS.png')
plt.show()